<a href="https://colab.research.google.com/github/Ranjani94/Reinforcement_Learning/blob/main/DL_training_Homework/Initialization_and_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [28]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [29]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [30]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Applying He Initialization

In [31]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        nn.init.kaiming_normal(self.fc.weight, mode='fan_in')
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [32]:
pip install pywick

In [33]:
pip install qhoptim

## Applying Nadam Optimizer using facebook qhoptim: Quasi-hyperbolic optimization which implements nadam optimizer

In [34]:
from torch.optim.optimizer import Optimizer
from qhoptim.pyt import QHM, QHAdam
import qhoptim
import pywick

In [35]:
model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# optimizer = pywick.optimizers.nadam.Nadam(model.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, schedule_decay=0.004)

optimizer = qhoptim.pyt.QHAdam(model.parameters(), lr=0.001, betas=(0.9, 0.999), nus=(1.0, 1.0), weight_decay=0.0, decouple_weight_decay=False, eps=1e-08)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  app.launch_new_instance()


In [36]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2493
Epoch [1/5], Step [200/600], Loss: 0.1679
Epoch [1/5], Step [300/600], Loss: 0.0609
Epoch [1/5], Step [400/600], Loss: 0.0469
Epoch [1/5], Step [500/600], Loss: 0.0834
Epoch [1/5], Step [600/600], Loss: 0.0637
Epoch [2/5], Step [100/600], Loss: 0.0705
Epoch [2/5], Step [200/600], Loss: 0.0623
Epoch [2/5], Step [300/600], Loss: 0.0210
Epoch [2/5], Step [400/600], Loss: 0.0845
Epoch [2/5], Step [500/600], Loss: 0.0237
Epoch [2/5], Step [600/600], Loss: 0.1987
Epoch [3/5], Step [100/600], Loss: 0.0506
Epoch [3/5], Step [200/600], Loss: 0.0479
Epoch [3/5], Step [300/600], Loss: 0.0496
Epoch [3/5], Step [400/600], Loss: 0.0637
Epoch [3/5], Step [500/600], Loss: 0.0159
Epoch [3/5], Step [600/600], Loss: 0.1235
Epoch [4/5], Step [100/600], Loss: 0.0357
Epoch [4/5], Step [200/600], Loss: 0.0237
Epoch [4/5], Step [300/600], Loss: 0.0183
Epoch [4/5], Step [400/600], Loss: 0.0188
Epoch [4/5], Step [500/600], Loss: 0.0310
Epoch [4/5], Step [600/600], Loss:

In [37]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.03 %
